# 패키지 불러오기

In [41]:
#TabNetClassifier, TabNetRegressor, TabNetMultiTaskClassifier 사용 가능
from pytorch_tabnet.tab_model import TabNetRegressor

import pandas as pd
import numpy as np
np.random.seed(0)

import torch
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

import os
import wget
from pathlib import Path

import matplotlib.pyplot as plt
%matplotlib inline

# 모델 설정

In [42]:
Dataset_name = 'TN_Baseline' #실험명

n_splits = 5 #교차검증 데이터셋 개수

target = '연봉' #목표변수(종속변수)

unused_feat = [] #사용하지 않는 feature 지정

#반복학습 데이터 전체를 1회 사용 = 1 epoch
max_epochs = 1000 if not os.getenv("CI", False) else 2 #max_epochs 기본값은 1000, 환경변수 "CI" 값이 True라면 max_epochs = 2 (CI값이 존재하지 않는 경우에도 False반환)

# 야구선수 데이터
- 경로는 사용자에 따라서 변경

In [43]:
hitter = pd.read_csv('C:/Users/USER/Desktop/학교/4-2/창의적종합설계/7주차 모델링/타자(모델링용).csv')
pitcher = pd.read_csv('C:/Users/USER/Desktop/학교/4-2/창의적종합설계/7주차 모델링/투수(모델링용).csv')

# 변수 설정
- 독립변수, 종속변수 설정
- 범주형 변수 인코딩

In [44]:
def Xy_split(dataset):
    nunique = dataset.nunique() #트레인데이터 내 고유값의 수
    types = dataset.dtypes #각 컬럼의 데이터 타입 반환

    categorical_columns = []
    categorical_dims =  {}
    cat_emb_dim = []
    
    for col in dataset.columns: #범주형 변수 지정
        if types[col] == 'object' or col in ['데뷔년도', '연도', 'FA여부', '출생연도', '출생월', '출생일', '팀명_KIA', '팀명_KT', '팀명_LG', '팀명_NC', '팀명_SK', '팀명_두산', '팀명_롯데', '팀명_삼성', '팀명_우리/히어로즈/넥센/키움', '팀명_한화', '팀명_현대', '포지션(수비)_1루수', '포지션(수비)_2루수', '포지션(수비)_3루수', '포지션(수비)_수비기록없음', '포지션(수비)_우익수', '포지션(수비)_유격수', '포지션(수비)_좌익수', '포지션(수비)_중견수', '포지션(수비)_포수']:
            cat_emb_dim.append(dataset[col].nunique())
            print(col, dataset[col].nunique()) #해당 변수와 중복되지 않은 값 개수 출력
            l_enc = LabelEncoder() #컬럼마다 번호를 매김
            dataset[col] = dataset[col].fillna("VV_likely") #빈셀에 VV_likely채움, 왜..?
            dataset[col] = l_enc.fit_transform(dataset[col].values) #dataset[col]에 따라 인코딩
            categorical_columns.append(col) #해당 변수 이름 저장
            categorical_dims[col] = len(l_enc.classes_) #해당 변수와 value 개수 저장

    
    features = [ col for col in dataset.columns if col not in unused_feat+[target]] #feature는 unused_feat와 목표변수를 제외하고 선정
 
    #categorical_columns = 범주형 컬럼, i는 순서, f는 컬럼명, 사용하는 feature 중 범주형컬럼이 몇번째 컬럼인지 저장
    cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]

    #범주형컬럼의 값의 종류의 개수를 저장
    cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

    X = dataset[features]
    y = dataset[target]
    
    return cat_dims, cat_emb_dim, cat_idxs, X, y

In [45]:
def TN_model(data_x, data_y):

    # TN 모델링
    performance = []
    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)

    for tr_idx, val_idx in cv.split(data_x):  

        X_train = data_x.iloc[tr_idx, :]
        y_train = data_y[tr_idx]

        X_valid = data_x.iloc[val_idx, :]
        y_valid = data_y[val_idx]

        X_train = X_train.values
        X_valid= X_valid.values
        y_train = y_train.values.reshape(-1, 1)
        y_valid = y_valid.values.reshape(-1, 1)
        
        #TabNetRegressor, 네트워크 파라미터
        reg = TabNetRegressor(cat_dims=cat_dims, cat_emb_dim=cat_emb_dim, cat_idxs=cat_idxs)
        #모델 피팅
        reg.fit(
            X_train=X_train, y_train=y_train,
            eval_set=[(X_train, y_train)],
            eval_name=['train'],
            eval_metric=['rmsle', 'mae', 'rmse', 'mse'], #평가 방식
            max_epochs=max_epochs, #반복 학습 횟수
            patience=50, #더이상 성능이 증가하지 않는 epoch를 얼마나 허용할지
            batch_size=1024, virtual_batch_size=128, #batch size = 한번에 학습하는 사이즈
            num_workers=0, #몇개의 CPU코어를 이용해 학습할지
            drop_last=False #True일 때, 불완전한 마지막 셔플을 학습에 참여하지 않게 함
        )

        pred_valid = reg.predict(X_valid)
        rmse = mean_squared_error(y_valid, pred_valid, squared=False)
        print(rmse)
        performance.append(rmse)
        
    # TN 성능종합
    performance.append(np.mean(performance))

    output = pd.DataFrame({Dataset_name:performance}, index=['cv1','cv2','cv3','cv4','cv5','평균'])
    return output

# train, test분할 후 모델링

#### 1. 타자

In [46]:
# train, test 분할
cat_dims, cat_emb_dim, cat_idxs, X, y = Xy_split(hitter)

데뷔년도 35
연도 19
FA여부 2
출생연도 37
출생월 12
출생일 31


In [47]:
# 타자데이터 도출
hitter_performance = TN_model(X, y)

Device used : cpu
epoch 0  | loss: 1312411333.39759| train_rmsle: 65.89123| train_mae: 15498.62447| train_rmse: 36226.35259| train_mse: 1312348622.28819|  0:00:00s
epoch 1  | loss: 1312354382.03374| train_rmsle: 56.99345| train_mae: 15496.5125| train_rmse: 36224.97083| train_mse: 1312248511.81687|  0:00:01s
epoch 2  | loss: 1312280092.06747| train_rmsle: 50.14078| train_mae: 15492.24502| train_rmse: 36221.75791| train_mse: 1312015745.91824|  0:00:02s
epoch 3  | loss: 1312174739.73976| train_rmsle: 45.10166| train_mae: 15485.30312| train_rmse: 36216.86199| train_mse: 1311661092.68512|  0:00:03s
epoch 4  | loss: 1312074636.18313| train_rmsle: 42.00782| train_mae: 15478.67048| train_rmse: 36210.11961| train_mse: 1311172762.44023|  0:00:04s
epoch 5  | loss: 1311943195.91325| train_rmsle: 41.29392| train_mae: 15480.20894| train_rmse: 36210.02329| train_mse: 1311165786.73642|  0:00:05s
epoch 6  | loss: 1311822190.11084| train_rmsle: 39.76093| train_mae: 15478.84178| train_rmse: 36207.82073| 

#### 2. 투수

In [48]:
# train, test 분할
cat_dims, cat_emb_dim, cat_idxs, X, y = Xy_split(pitcher)

데뷔년도 35
연도 19
FA여부 2
출생연도 38
출생월 12
출생일 31


In [49]:
# 투수데이터 도출
pitcher_performance = TN_model(X, y)

Device used : cpu
epoch 0  | loss: 691787015.04928| train_rmsle: 65.51295| train_mae: 11372.49178| train_rmse: 26301.61142| train_mse: 691774763.12242|  0:00:01s
epoch 1  | loss: 691765436.66087| train_rmsle: 59.74959| train_mae: 11371.54195| train_rmse: 26301.0532| train_mse: 691745399.59179|  0:00:02s
epoch 2  | loss: 691744199.7913| train_rmsle: 53.4552 | train_mae: 11369.69738| train_rmse: 26299.51259| train_mse: 691664362.23657|  0:00:04s
epoch 3  | loss: 691711743.62899| train_rmsle: 47.20609| train_mae: 11366.71953| train_rmse: 26298.31131| train_mse: 691601177.75981|  0:00:05s
epoch 4  | loss: 691673538.78261| train_rmsle: 41.05944| train_mae: 11360.99088| train_rmse: 26295.24451| train_mse: 691439884.08499|  0:00:07s
epoch 5  | loss: 691605984.46377| train_rmsle: 35.64957| train_mae: 11351.61288| train_rmse: 26291.02821| train_mse: 691218164.10195|  0:00:09s
epoch 6  | loss: 691533731.24638| train_rmsle: 31.23054| train_mae: 11339.66087| train_rmse: 26284.11288| train_mse: 690

In [50]:
hitter_performance

,TN
cv1,13409.284670
cv2,10904.623674
cv3,12588.480078
cv4,16163.517816
cv5,12408.297991
평균,13094.840846


In [51]:
pitcher_performance

,TN
cv1,13042.099116
cv2,15112.128325
cv3,13040.088107
cv4,9923.675041
cv5,8758.584343
평균,11975.314986
